# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/3C%E5%AE%B6%E9%9B%BB/"  #3C家電

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='C:\\Users\\Rocky\\Downloads\\chromedriver.exe')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


國會必須介入以保護美國人作為消費者的需求和作為公民的權利。而在那之前，可以肯定的是，我們正生活在世界上最先進的監視系統中。
其他人也看了這些...
----------------------------------------------------------------------

[73] 拍照功能更強大！蘋果要推外部閃光配件　iPhone 11以上有支援

▲Lume Cube 創意燈光套件。（圖／翻攝自蘋果官網）
記者陳心怡／外電報導
市面上目前有一些帶有內建閃光燈的iPhone配件，例如Lume Cube創意燈光套件，Apple Store稱其為「有史以來第一個與iPhone相容的相機閃光燈」，但它是利用藍牙進行連接，無法運行MFi認證。Lume Cube還具有一個光學感測器，當iPhone的內建相機閃光燈啟動時，外部附屬閃光燈能同步偵測到。
另一個硬體配件叫做Tric，能讓外部的閃光燈與iPhone拍照鍵同步，但包括第三方應用程式在內，所有解決方案都必須透過藍牙才能同步進行。
上述功能僅iPhone 11，iPhone 11 Pro和iPhone 11 Pro Max等新一代機種有支援。
----------------------------------------------------------------------

[74] 地方媽媽爽當玩咖！私廚女王Irene忙育兒又工作　中華電信網路門市一鍵搞定

▲私廚女王Irene忙育兒又工作　中華電信網路門市一鍵搞定（影／ETtoday)
▲私廚女王Irene覺得讓身邊的人幸福就是一種成功。（圖／翻攝自影片，下同）
消費中心／綜合報導
下班後，若能跟同事好友聚在一起，吃點美食、吐吐苦水，一整天的工作壓力都抒發了，十分療癒，任職於媒體的Irene胡碧玲，正是因為喜歡下班後約朋友到家裡吃飯放鬆一下，沒想到自己在席間隨意發揮創意做料理招待朋友，竟讓好手藝口耳相傳到朋友圈和不認識的人都慕名而來，因此才動了做私廚的念頭，為了不忘了自製的各式創意餐點，她開設了FB粉專，記錄下每次的菜色和食客的回饋，私廚女王的稱號不逕而走，去年更出版《私廚女王Irene的快速料理》食譜書，除了平時的工作和因興趣而開始的私廚事業，Irene還是一名單親媽媽，有個還在就讀小學的兒子，生活被工作育兒塞滿滿的她，非常重視效率，透